# Cleaning Booking.com

In [ ]:
import csv
import pandas as pd
import re

## 1) check for duplicates

In [ ]:
with open('booking.csv') as booking_file:
    data = csv.reader(booking_file)

In [ ]:
# Open the input and output CSV files with the appropriate encoding
with open('booking.csv', 'r', encoding='utf-8') as input_file, open('booking_without_duplicates.csv', 'w', newline='', encoding='utf-8') as output_file:
    # Create a CSV reader and writer
    csv_reader = csv.reader(input_file)
    header_row = next(csv_reader)
    csv_writer = csv.writer(output_file)

    # Initialize a set to store encountered rows
    encountered_rows = set()

    # Read the input CSV data and write unique rows to the output CSV file
    for row in csv_reader:
        # Convert the row to a tuple for hashability
        row_tuple = tuple(row)

        # Check if the row is a duplicate
        if row_tuple not in encountered_rows:
            # Write the row to the output CSV file
            csv_writer.writerow(row)

            # Add the row to the set of encountered rows
            encountered_rows.add(row_tuple)


In [ ]:
# Count nomber of rows i the original csv file
with open('booking.csv', 'r', encoding='utf-8') as input_file:
    num_rows_original = sum(1 for _ in input_file)

# Count nomber of rows i the csv file without duplicates
with open('booking_without_duplicates.csv', 'r', encoding='utf-8') as output_file:
    num_rows_without_duplicates = sum(1 for _ in output_file)

# Print number of rows for each csv
print("Numero di righe nel file CSV originale: ", num_rows_original)
print("Numero di righe nel nuovo file CSV senza duplicati: ", num_rows_without_duplicates)


## 2) clean the data using RE

In [ ]:
# Create a list of non-duplicate rows
encountered_rows = list(encountered_rows)
# Create a DataFrame using the list
base = pd.DataFrame(encountered_rows, columns=header_row)
base

## Cleaning of the 'indirizzo' column

In [ ]:
base['indirizzo']

### Create 'stato' column

In [ ]:
# Split the column "indirizzo" where I find the last comma. The last word is the 'Stato' so I create the column "Stato" where I cancopy the name of the 'Stato'.
base['stato'] = base['indirizzo'].apply(lambda x: 'Italia' if ', Italia' in x else None)
# Drop the 'Stato' name from the 'indirizzo' column
base['indirizzo'] = base['indirizzo'].apply(lambda x: x.replace(', Italia', '').strip() if ', Italia' in x else x)

# Print the DataFrame
base[['indirizzo','stato']]

In [ ]:
# Check the 'comune' column
base[['indirizzo','comune','stato']]

### Create 'comune' column

In [ ]:
# Find the name of the 'comune' in 'indirizzo',
# if 'comune' is inside 'indirizzo' drop it from the 'indirizzo' column,
# else mantain the column as it was

base['indirizzo'] = base.apply(lambda row: row['indirizzo'].replace(row['comune'], '').strip() if row['comune'] in row['indirizzo'] else row['indirizzo'], axis=1)

base[['indirizzo','comune','stato']]

### Create 'CAP' column

In [ ]:
# extract the CAP from 'indirizzo' column and create a new column 'CAP' to insert it

def extract_cap(indirizzo):
    cap_match = re.search(r'\b\d{5}\b', indirizzo)
    if cap_match:
        cap = cap_match.group()
        return cap
    return None

base['CAP'] = base['indirizzo'].apply(extract_cap)
base['indirizzo'] = base['indirizzo'].apply(lambda x: re.sub(r'\b\d{5}\b', '', x).strip())

# drop all the commas at the end of the string in 'indirizzo'
base['indirizzo'] = base['indirizzo'].apply(lambda x: x.rstrip(',').strip())

base[['indirizzo','comune','stato', 'CAP']]

### Split the dataset in two: rows_with_number, rows_without_number

In [ ]:
# check in the column "indirizzo" all the strings that starts with a number and all the sub_dataframe "rows_with_number"
def check_numero(indirizzo):
    parole = indirizzo.split() # split 'indirizzo' in words using the space as separator
    if len(parole) > 0: # if the cell is not empty
        first_word = parole[0] # assign the first word at the 'first_word' variable
        return re.match(r'^\d+$', first_word) is not None # check if the first word is an integer number
    return False # if the length of the word is 0 or the corrispondence doesn't found return False

base['first_word_number'] = base['indirizzo'].apply(check_numero)

In [ ]:
# The first word is a number
rows_with_number = base.loc[base['first_word_number']]

rows_with_number[['indirizzo', 'first_word_number']]

In [ ]:
# The first word is not a number
rows_without_number = base.loc[~base['first_word_number']]

rows_without_number[['indirizzo', 'first_word_number']]

### Work on the first sub-dataset: rows_with_number

In [ ]:
# check if in the column 'indirizzo' there are more than 4 words.
# If there are more than 4 words split the string after the fourth and copy the last
# in a new column named 'info_indirizzo'

def modify_address(row):
    indirizzo = row['indirizzo']
    parole = indirizzo.split()

    if len(parole) > 4:
        info_indirizzo = ' '.join(parole[4:])
        indirizzo = ' '.join(parole[:4])
    else:
        info_indirizzo = ''

    row['info_indirizzo'] = info_indirizzo
    row['indirizzo'] = indirizzo
    return row

rows_with_number = rows_with_number.apply(modify_address, axis=1)

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# drop duplicates between column 'indirizzo' and column 'info_indirizzo'

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if indirizzo == info_indirizzo:
        info_indirizzo = ''

    rows_with_number.at[index, 'info_indirizzo'] = info_indirizzo

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Imposta l'opzione per visualizzare tutte le colonne
# pd.set_option('display.max_columns', None)

# Imposta l'opzione per visualizzare tutte le righe
# pd.set_option('display.max_rows', None)


# rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for some particular words in the 'indirizzo' column

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    # check if string '2nd' is in 'indirizzo'
    if '2nd' in indirizzo.lower():
        info_indirizzo = '2nd ' + info_indirizzo
        indirizzo = indirizzo.replace('2nd', '').strip()

    # check if string 'Int.' is in 'indirizzo'
    if 'Int.' in indirizzo:
        info_indirizzo = 'Int. ' + info_indirizzo
        indirizzo = indirizzo.replace('Int.', '').strip()

    rows_with_number.at[index, 'info_indirizzo'] = info_indirizzo
    rows_with_number.at[index, 'indirizzo'] = indirizzo

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for some particular words in the 'info_indirizzo' column

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    key_words = ['piano', 'floor', 'scala']
    key_found = [words for words in key_words if words.lower() in info_indirizzo.lower()]

    if key_found:
        # if one of the key word is present go on
        rows_with_number.at[index, 'info_indirizzo'] = info_indirizzo
    else:
        # if there are no key words in 'info_indirizzo', copy and paste 'info_indirizzo' at the end of 'indirizzo'
        rows_with_number.at[index, 'indirizzo'] = indirizzo + ' ' + info_indirizzo
        rows_with_number.at[index, 'info_indirizzo'] = ''

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for some particular words in the 'indirizzo' or 'info_indirizzo' columns

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    # check for "Mascari" in "info_indirizzo"
    if 'mascari' in info_indirizzo.lower():
        indirizzo = indirizzo + ' Mascari'
        info_indirizzo = info_indirizzo.replace('Mascari', '').strip()

    # check for "4A" in "indirizzo"
    if 'terra 4A' in indirizzo:
        indirizzo = indirizzo.replace('terra 4A', '').strip()
        info_indirizzo = info_indirizzo + 'terra 4A'

    # check for "terra - Condominio "Eleonora A"" in "indirizzo"
    if 'terra - Condominio "Eleonora A"' in indirizzo:
        indirizzo = indirizzo.replace('terra - Condominio "Eleonora A"', '').strip()
        info_indirizzo = info_indirizzo + 'terra - Condominio "Eleonora A"'

    # check for "Condominio Ginepro" in "indirizzo"
    if 'Condominio Ginepro' in indirizzo:
        indirizzo = indirizzo.replace('Condominio Ginepro', '').strip()
        info_indirizzo = info_indirizzo + 'Condominio Ginepro'

    # check for "primo" in "indirizzo"
    if 'primo' in indirizzo:
        indirizzo = indirizzo.replace('primo', '').strip()
        info_indirizzo = info_indirizzo + 'primo piano'

    # check for "frazione Primolo" in "indirizzo"
    if 'frazione Primolo' in indirizzo:
        indirizzo = indirizzo.replace('frazione Primolo', '').strip()
        info_indirizzo = info_indirizzo + 'frazione Primolo'

    rows_with_number.at[index, 'indirizzo'] = indirizzo
    rows_with_number.at[index, 'info_indirizzo'] = info_indirizzo

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for some particular symbols in the 'indirizzo' column

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']

    # Remove unnecessary spaces at the end of the string
    indirizzo = indirizzo.rstrip()

    # Remove double spaces and replace them with a single space
    indirizzo = ' '.join(indirizzo.split())

    # Remove the symbol " -" from 'indirizzo'
    indirizzo = indirizzo.replace(" -", " ")

    # Checking for repetitions of words or numbers
    parole_numeri = indirizzo.split()
    parole_numeri_unici = []
    for parola_numero in parole_numeri:
        if parola_numero not in parole_numeri_unici:
            parole_numeri_unici.append(parola_numero)

    # Address reconstruction with unique words or numbers
    indirizzo = ' '.join(parole_numeri_unici)

    rows_with_number.at[index, 'indirizzo'] = indirizzo

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Move numbers at the end of 'indirizzo' column

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']

    # Split the address string into words
    parole = indirizzo.split()

    # Extract number from addresses
    numero = parole.pop(0)

    # Rebuild address with desired format
    indirizzo = ' '.join(parole) + ' ' + numero

    rows_with_number.at[index, 'indirizzo'] = indirizzo

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for letter + dot in the 'indirizzo' column

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']

    # Check for a letter followed by a dot in 'indirizzo'
    if re.search(r'[A-Za-z]\.', indirizzo):
        print(indirizzo)

In [ ]:
# Mapping abbreviated names

nominativi_abbreviati = {
    'G.Garibaldi': 'Giuseppe Garibaldi',
    'G. Carducci': 'Giosuè Carducci',
    'C. Battisti': 'Cesare Battisti',
    'Don P. Mariani': 'Don Paolo Mariani',
    'G. Osio': 'Generale Osio'
}

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']

    # Controlla se l'indirizzo contiene una lettera seguita da un punto
    if re.search(r'[A-Za-z]\.', indirizzo):
        # Sostituisci i nomi abbreviati con i nomi completi
        for abbreviato, completo in nominativi_abbreviati.items():
            indirizzo = re.sub(r'\b' + re.escape(abbreviato) + r'\b', completo, indirizzo)
        # Aggiorna il valore dell'indirizzo nel DataFrame
        rows_with_number.at[index, 'indirizzo'] = indirizzo

rows_with_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for 'loc' in the 'indirizzo' column

b = []

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if 'loc' in str(indirizzo).lower():
        b.append(row)
df_b = pd.DataFrame(b)

df_b

In [ ]:
# check for 'località' in the 'indirizzo' column

rows_with_number['località'] = ''

for index, row in rows_with_number.iterrows():
    indirizzo = row['indirizzo']

    # Controllo se l'indirizzo contiene la parola "località"
    if 'località' in indirizzo.lower():
        # Sposto tutto il contenuto della cella nella colonna "località"
        rows_with_number.at[index, 'località'] = indirizzo
        rows_with_number.at[index, 'indirizzo'] = ''

rows_with_number[['indirizzo', 'località']]

In [ ]:
rows_with_number

## Update the starting Dataset named "base" with rows_with_number

In [ ]:
# Creating a temporary DataFrame for selected rows.
selected_rows = base.copy()

# Adding 'info_address' and 'location' columns to the temporary DataFrame.
selected_rows['info_indirizzo'] = ''
selected_rows['località'] = ''

# Overwrite selected rows in the base DataFrame.
base = pd.concat([base, selected_rows])

# Printing the updated 'base' DataFrame
base

In [ ]:
# Creating a temporary DataFrame for selected rows.
selected_rows = rows_with_number[['indirizzo', 'info_indirizzo', 'località']]

# Overwriting selected rows in the base DataFrame.
base.update(selected_rows)

# Print the updated base DataFrame.
base

### Work on the second sub-dataset: rows_without_number

In [ ]:
# check for a specific pattern

pattern = r'\s+'

for index in rows_without_number.index:
    indirizzo = rows_without_number.loc[index, 'indirizzo']

    indirizzo = indirizzo.replace(',', '').strip()
    indirizzo = indirizzo.replace(pattern, ' ').strip()

    rows_without_number.loc[index, 'indirizzo'] = indirizzo

rows_without_number['indirizzo']

In [ ]:
# Make some specific substitutions in the 'address' column

rows_without_number['info_indirizzo'] = ''

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']

    # check for "via 25 aprile" in "indirizzo"
    if 'via 25 aprile' in indirizzo.lower():
        indirizzo = indirizzo.replace('via 25 aprile', 'Via XXV Aprile').strip()

    # check for "4 novembre" in "indirizzo" (uso le regex perchè altrimenti non funziona)
    if re.search(r'\b4 novembre\b', indirizzo, flags=re.IGNORECASE):
        indirizzo = re.sub(r'\b4 novembre\b', 'IV Novembre', indirizzo, flags=re.IGNORECASE).strip()

    rows_without_number.at[index, 'indirizzo'] = indirizzo

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Function to divide the 'indirizzo'

def divide_indirizzo(row):
    regex = r'^(.*?\b\d{1,4}\b)(.*)$'
    matches = re.match(regex, row['indirizzo'])
    if matches:
        row['info_indirizzo'] = matches.group(2).strip()
        row['indirizzo'] = matches.group(1).strip()
    else:
        row['info_indirizzo'] = ''
    return row

# Apply the divide_address function to each row in the DataFrame.
rows_without_number = rows_without_number.apply(divide_indirizzo, axis=1)

# Print DataFrame
rows_without_number[['indirizzo', 'info_indirizzo']]


In [ ]:
# Remove "n." from 'indirizzo' (es. via Roma 70)
rows_without_number['indirizzo'] = rows_without_number['indirizzo'].str.replace(r'n\.', '', flags=re.IGNORECASE, regex=True)

# Substitute "SS 301", "Via SS. 301", "SS301", "Via S.S. 301", "Via Statale 301" e "S.s 301" with "SS301" in 'indirizzo'
rows_without_number['indirizzo'] = rows_without_number['indirizzo'].str.replace(r'SS\s*301|Via\s*(?:SS\.|S\.S\.)\s*301|S\.s\s*301|Via\s*Statale\s*301', 'SS301', flags=re.IGNORECASE, regex=True)

# Substitute "Via Strada Statale 6763" with "SS301 6763" in 'indirizzo'
rows_without_number['indirizzo'] = rows_without_number['indirizzo'].str.replace('Via Strada Statale 6763', 'SS301 6763', flags=re.IGNORECASE, regex=False)

# Print dataframe
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Filtering rows where there is a single letter or a letter preceded/followed by a symbol in 'info_address'

#rows_without_number['info_indirizzo'] = rows_without_number[rows_without_number['info_indirizzo'].str.contains(r'(^|\W)[A-Za-z](\W|$)')]
#rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Remove all the dots in the 'indirizzo' column and 'info_indirizzo' column
rows_without_number['info_indirizzo'] = rows_without_number['info_indirizzo'].str.replace('.', '', regex=True)

# Print dataframe
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for some particular patterns in the 'info_indirizzo' column and 'indirizzo' column

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    # Search for matches of groups consisting of a single letter or a single letter preceded by '/'
    matches = re.findall(r'(^|\s)(/?[A-Za-z])\b', info_indirizzo)

    # Add the corrispondences found at the end of 'indirizzo'
    for match in matches:
        indirizzo += match[1]

    # Remove the corrispondences from 'info_indirizzo'
    info_indirizzo = re.sub(r'(^|\s)(/?[A-Za-z])\b', '', info_indirizzo)

    # Update the values in 'indirizzo' e 'info_indirizzo'
    rows_without_number.at[index, 'indirizzo'] = indirizzo.strip()
    rows_without_number.at[index, 'info_indirizzo'] = info_indirizzo.strip()

# Remove eventual duplicates of spaces in 'indirizzo'
rows_without_number['indirizzo'] = rows_without_number['indirizzo'].str.replace(r'\s+', ' ', regex=True)

# Print DataFrame
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
rows_without_number[rows_without_number['indirizzo'] == 'via scleva']

In [ ]:
# check for some particular patterns in the 'info_indirizzo'

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    # check if string 'via' is in 'info_indirizzo'
    if 'via scleva 5' in indirizzo.lower():
        indirizzo = indirizzo.replace(indirizzo, info_indirizzo).strip()
        # Elimina la stringa dalla colonna 'info_indirizzo'
        row['info_indirizzo'] = ''

    # Aggiorna il valore nella colonna 'indirizzo'
    rows_without_number.at[index, 'indirizzo'] = indirizzo
    rows_with_number.at[index, 'info_indirizzo'] = info_indirizzo

# Stampa del DataFrame risultante
rows_without_number[['indirizzo', 'info_indirizzo']]

### VISUALIZATION

In [ ]:
# check for some particular patterns in the 'info_indirizzo' column and 'indirizzo' column

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if info_indirizzo != '':
        # Esegui la sostituzione nella colonna 'info_indirizzo' utilizzando la funzione re.sub
        rows_without_number.at[index, 'info_indirizzo'] = re.sub(r'([-/])\s*(\d+)', r'/\2', info_indirizzo)

    # Aggiorna il valore nella colonna 'indirizzo'
    rows_without_number.at[index, 'indirizzo'] = indirizzo

# Stampa le righe filtrate con le colonne 'indirizzo' e 'info_indirizzo'
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Filter rows that contain numbers or number blocks without brackets in the 'info_address' column.
non_empty_rows = rows_without_number[rows_without_number['info_indirizzo'].notnull() & (rows_without_number['info_indirizzo'] != '')]

non_empty_rows[['indirizzo', 'info_indirizzo']]

In [ ]:
# Filter rows that contain the pattern "/number" in the "info_address" column.
pattern = r"/\d+"  # Il pattern di ricerca è "/numero", dove "numero" rappresenta una o più cifre
matching_rows = rows_without_number[rows_without_number['info_indirizzo'].str.contains(pattern, regex=True, na=False)]

matching_rows[['indirizzo', 'info_indirizzo']]

In [ ]:
# Function for pattern append and removal from info_address column.
def append_and_remove_pattern(row):
    pattern = r"(/(\d+))"  # The search pattern is "/number," where "number" represents one or more digits
    info_indirizzo = row['info_indirizzo']
    indirizzo = row['indirizzo']

    matches = re.findall(pattern, info_indirizzo)
    for match in matches:
        pattern_match = match[0]  # Get the corresponding pattern, including the "/" sign.
        indirizzo += pattern_match  # Add pattern to existing address.
        info_indirizzo = info_indirizzo.replace(pattern_match, '')  # Remove pattern from info_address column.

    # Update the cells "address" and "info_address" in the original DataFrame.
    row['indirizzo'] = indirizzo
    row['info_indirizzo'] = info_indirizzo

    return row

# Applies the function to each row in the DataFrame and assigns the result to the original DataFrame.
rows_without_number = rows_without_number.apply(append_and_remove_pattern, axis=1)

# Print dataframe
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
matching_rows[['indirizzo', 'info_indirizzo']]

In [ ]:
# Filter rows that contain numbers or number blocks without brackets in the 'info_address' column.
non_empty_rows = rows_without_number[rows_without_number['info_indirizzo'].notnull() & (rows_without_number['info_indirizzo'] != '')]

non_empty_rows[['indirizzo', 'info_indirizzo']]

In [ ]:
# Error: there was a SS301N instead of SS301, so we replace it.
rows_without_number['indirizzo'] = rows_without_number['indirizzo'].replace('SS301N', 'SS301')
rows_without_number[rows_without_number['indirizzo'].str.contains('SS301')]

# Print lines that still contain the original string
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Find the lines that contain the searched phrase.
rows_containing_phrase = rows_without_number[rows_without_number['indirizzo'].str.contains('statale', case=False)]

rows_containing_phrase

In [ ]:
# Error: there was a state highway 301 instead of SS301, so we replace it.
# Create a Boolean mask to locate the rows that contain the searched phrase.
mask = rows_without_number['indirizzo'].str.contains('via strada statale 301', case=False)

# Replace the phrase found with 'SS301'
rows_without_number.loc[mask, 'indirizzo'] = 'SS301'

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check if it is correct
rows_with_ss301 = rows_without_number[rows_without_number['indirizzo'].str.contains('SS301')]

print(rows_with_ss301[['indirizzo', 'info_indirizzo']])

In [ ]:
# Now we look to see if there is any numeric value in the "info_address" column, if there is we move it to the "address" column with a space after SS301

# Select the rows that have "SS301" in the "address" column.
righe_SS301 = rows_without_number[rows_without_number['indirizzo'].str.contains('SS301')]

# Extract the numeric value from the "info_address" column of the selected rows
righe_SS301['numero'] = righe_SS301['info_indirizzo'].str.extract(r'(\d+)')

# Move the numeric value to the "address" column with a space after SS301
righe_SS301['indirizzo'] = righe_SS301['indirizzo'].str.replace('SS301', 'SS301 ', regex=False) + righe_SS301['numero'].fillna('')

# Remove only numeric values from "info_address" column of selected rows.
righe_SS301['info_indirizzo'] = righe_SS301['info_indirizzo'].mask(righe_SS301['numero'].notnull(), '')

# Update the dataframe "address_with_space" with the modified rows.
rows_without_number.update(righe_SS301)


rows_without_number[rows_without_number['indirizzo'].str.contains('SS301')]

In [ ]:
filtered_rows = rows_without_number[rows_without_number['info_indirizzo'].str.contains(r'(?i)\bvia\b')]

filtered_rows[['indirizzo', 'info_indirizzo']]

In [ ]:
# Line processing function

def process_row(row):
    info_indirizzo = row['info_indirizzo']
    indirizzo = row['indirizzo']

    # Checks whether the line contains the word "street" regardless of case difference
    if re.search(r'(?i)\bvia\b', info_indirizzo):
        # Check if the line contains brackets.
        if re.search(r'\(.*\)', info_indirizzo):
            # Remove everything before the parentheses from the address
            indirizzo = re.sub(r'.*\(', '', indirizzo)
              # Remove everything following the round bracket from the info_address line.
            info_indirizzo = re.sub(r'\(.*\)', '', info_indirizzo)
        else:
            # get all the text from the info_address line
            text_to_append = info_indirizzo.strip()
            # Remove words, numbers or symbols already in the address
            text_to_append = re.sub(r'\b\w+\b', '', text_to_append)
            text_to_append = re.sub(r'\d+', '', text_to_append)
            text_to_append = re.sub(r'\W+', '', text_to_append)
            # Add text to address
            indirizzo += text_to_append
            # Remove text from info_address line
            info_indirizzo = info_indirizzo.replace(text_to_append, '')

    # Update the cells "address" and "info_address" in the original DataFrame.
    row['indirizzo'] = indirizzo
    row['info_indirizzo'] = info_indirizzo

    return row

# Applies the function to each row in the DataFrame and assigns the result to the original DataFrame
rows_without_number = rows_without_number.apply(process_row, axis=1)

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for particular strings in the column 'indirizzo'

v = ["Via Serta", "Via Falck", "Via al Forte Santa Lucia"]

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    for via in v:
        if via in info_indirizzo:
            indirizzo = info_indirizzo + " " + indirizzo
            info_indirizzo = info_indirizzo.replace(via, "").strip()

    if 'via Scleva 5/C' in info_indirizzo:
        info_indirizzo = info_indirizzo.replace('via Scleva 5/C', "").strip()

    if 'Check-in' in indirizzo:
        info_indirizzo = indirizzo + "" + info_indirizzo
        indirizzo = indirizzo.replace('In varie zone di (Check-in: Via Saroch 606', "").strip()

    # Update the values
    rows_without_number.at[index, 'indirizzo'] = indirizzo
    rows_without_number.at[index, 'info_indirizzo'] = info_indirizzo

# Stampa le righe filtrate con le colonne 'indirizzo' e 'info_indirizzo'
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
def sostituisci_localita(parola):
    parola = parola.lower()  # Convert word to lower case for searching
    parola = parola.replace("località", "località")  # Replace "località" with "località"
    parola = parola.replace("localitá", "località")  # Replace "localitá" with "località"
    parola = parola.replace("localita'", "località")  # Replace "localita'" with "località"
    parola = parola.replace("loc.", "località")  # Replace "loc." with "località"
    return parola

# Apply replacement to 'address' and 'info_address' columns.
rows_without_number['indirizzo'] = rows_without_number['indirizzo'].apply(sostituisci_localita)
rows_without_number['info_indirizzo'] = rows_without_number['info_indirizzo'].apply(sostituisci_localita)

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# List of words that have to be excluded
parole_escluse = ["via", "piazza", "viale", "corso", "vicolo", "piazzale", "SS301", "località", "contrada", "salita", "campo"]

# FUnction to verify the words
def check_parole_escluse(row):
    indirizzo = row['indirizzo']

    # Check if 'indirizzo' doesn't contain excluded words, ignoring uppercase and lowercase
    for parola in parole_escluse:
        if re.search(rf'\b{parola}\b', indirizzo, flags=re.IGNORECASE):
            return False

    return True

# Filter the rows that satisfies the condition
righe_filtrate = rows_without_number[rows_without_number.apply(check_parole_escluse, axis=1)]

righe_filtrate[['indirizzo', 'info_indirizzo']]

In [ ]:
#case sparse del tonale
#strada per scilano
#galizzi
#bormio 2000
#fraz. barzesto 5/d
#trepalle
#p.zza s. abbondio 3
#corso giuseppe zanardelli 15

In [ ]:
# List of words to find
parole_da_cercare = ["saroch", "quattro porte", "trento", "delle soste", "conciliazione", "nuova",
                     "fratelli lazzaroni", "prevalè", "prato valentino", "generale osio", "marsala",
                     "bernina", "predalunga", "renso"]

def modifica_indirizzo(row):
    indirizzo = row['indirizzo']

    for parola in parole_da_cercare:
        pattern = rf"\b{parola}\b"  # Parola completa come parola intera
        if "via" not in indirizzo:
            if re.search(pattern, indirizzo, flags=re.IGNORECASE):
                indirizzo = re.sub(pattern, f"via {parola}", indirizzo, flags=re.IGNORECASE)

    return indirizzo

rows_without_number['indirizzo'] = rows_without_number.apply(modifica_indirizzo, axis=1)

rows_without_number

In [ ]:
# Functio to modify 'indirizzo'
def modifica_indirizzo(row):
    indirizzo = row['indirizzo']

    # Modify "zanardelli 15" in "corso zanardelli 15"
    indirizzo = re.sub(r'\bzanardelli 15\b', 'corso zanardelli 15', indirizzo, flags=re.IGNORECASE)

    # Modify "p.zza s. abbondio 3" in "piazza sant'abbondio 3"
    indirizzo = re.sub(r'\bp\.zza s\. abbondio 3\b', "piazza sant'abbondio 3", indirizzo, flags=re.IGNORECASE)

    return indirizzo

rows_without_number['indirizzo'] = rows_without_number.apply(modifica_indirizzo, axis=1)

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# List of words that have to be excluded
parole_escluse = ["via", "piazza", "viale", "corso", "vicolo", "piazzale", "SS301", "località", "contrada", "salita", "campo"]

def check_parole_escluse(row):
    indirizzo = row['indirizzo']

    # Check if 'indirizzo' doesn't contain excluded words, ignoring uppercase and lowercase
    for parola in parole_escluse:
        if re.search(rf'\b{parola}\b', indirizzo, flags=re.IGNORECASE):
            return False

    return True

rows_without_number_filtered = rows_without_number[rows_without_number.apply(check_parole_escluse, axis=1)]

rows_without_number_filtered[['indirizzo', 'info_indirizzo']]

In [ ]:
# check for bormio 2000

def sposta_contenuto(row):
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if info_indirizzo == '' and indirizzo != "bormio 2000":
        info_indirizzo = indirizzo
        indirizzo = ''

    return pd.Series({'indirizzo': indirizzo, 'info_indirizzo': info_indirizzo})

rows_without_number_filtered[['indirizzo', 'info_indirizzo']] = rows_without_number_filtered.apply(sposta_contenuto, axis=1)

rows_without_number_filtered[['indirizzo', 'info_indirizzo']]

In [ ]:
rows_without_number.loc[rows_without_number_filtered.index] = rows_without_number_filtered

In [ ]:
rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# Substitution of some words

def substitute_words(word):
    word = word.lower()  # Converti la parola in minuscolo per la ricerca
    word = word.replace("n°", "")
    word = word.replace("s. pellico", "silvio pellico")
    word = word.replace("e. de gasperi", "enzo de gasperi")
    word = word.replace("s. silvestro", "san silvestro")
    word = word.replace("g. carducci", "giosuè carducci")
    word = word.replace("g. marconi", "guglielmo marconi")
    word = word.replace("f. manzoni", "francesca manzoni")
    word = word.replace("i. de giacomi", "innocenti de giacomi")
    word = word.replace("v. bonomelli ", "vittorio bonomelli ")
    word = word.replace("via dosde' 20", "via dosdè 20")
    word = word.replace("via prevale' 2b", "via prevalè 2b")
    word = word.replace("s. antonio", "sant'antonio")
    word = word.replace("s.francesco", "san francesco")
    word = word.replace("dott.", "dottor")
    word = word.replace("f.lli", "fratelli")
    word = word.replace("s. pietro", "san pietro")
    word = word.replace("s. nicolò", "san nicolò")
    word = word.replace("sant antonio", "sant'antonio")
    word = word.replace("via renso’ 95", "via rensò 95")
    word = word.replace("fraz.", "frazione")
    word = word.replace('via turte6 2', 'via turte 6 2')
    word = word.replace('via funivia / zandilla 6', 'via zandilla 6')
    return word


rows_without_number['indirizzo'] = rows_without_number['indirizzo'].apply(substitute_words)
rows_without_number['info_indirizzo'] = rows_without_number['info_indirizzo'].apply(substitute_words)

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# I do a check of the errors I have displayed.
# ERROR 1) Some cells in the 'address' column still contain floor information: via donati ground floor,
# via parvina ground floor and via prato valentino teglio first floor number 12.

c = []

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if 'piano' in str(indirizzo).lower() and 'via' in str(indirizzo).lower():
        c.append(row)

df_c = pd.DataFrame(c)

df_c

In [ ]:
# keep only street and location in the 'address' column.

def piano(row):
    keyword = "piano"
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']  # Aggiunto per mantenere il contenuto esistente

    index = indirizzo.lower().find(keyword.lower())
    if index != -1:
        row['indirizzo'] = indirizzo[:index].strip()
        # Concatenation of information with the existing content in the 'info_indirizzo' column
        row['info_indirizzo'] = info_indirizzo + ' ' + indirizzo[index:].strip()
    else:
        row['info_indirizzo'] = info_indirizzo

    return row

rows_without_number = rows_without_number.apply(piano, axis=1)

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# ERROR 2) Some cells in the 'address' column have numbers at the beginning followed by letters: 14h via cardo rialzato,
# 66a via san nicolò 2c, via milano.

c = []

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    pattern = r"(^\d+[A-Za-z])"

    matches = re.search(pattern, indirizzo)

    if matches:
        c.append(indirizzo)

df_c = pd.DataFrame(c)

df_c

In [ ]:
for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']

    # Use regex to match a numeric group followed by a letter at the beginning of the address
    match = re.match(r'(^\d+[A-Za-z])', indirizzo)

    if match:
        gruppo_numerico_lettera = match.group(1)

        # Remove the matched group from the address
        indirizzo = indirizzo.replace(gruppo_numerico_lettera, '')

        # Rebuild address with desired format
        indirizzo = indirizzo.strip() + ' ' + gruppo_numerico_lettera

    rows_without_number.at[index, 'indirizzo'] = indirizzo

rows_without_number[['indirizzo', 'info_indirizzo']]

In [ ]:
# ERROR 3) Some cells in the 'address' column have both street and location.

c = []

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if 'località' in str(indirizzo).lower() and 'via' in str(indirizzo).lower():
        c.append(row)

df_c = pd.DataFrame(c)

df_c

In [ ]:
# move località in a new column called 'località'

rows_without_number['località'] = ''

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']

    # Check if 'indirizzo' contains "via" and "località"
    if 'via' in indirizzo and 'località' in indirizzo:
        parole = indirizzo.split()

        # Check if 'via' is before "località"
        if parole.index('via') < parole.index('località'):
            # Split before "località"
            localita_index = parole.index('località')
            via_parte = ' '.join(parole[:localita_index])
            localita_parte = ' '.join(parole[localita_index:])
        else:
            # Split before "via"
            via_index = parole.index('via')
            via_parte = ' '.join(parole[via_index:])
            localita_parte = ' '.join(parole[:via_index])

        rows_without_number.at[index, 'indirizzo'] = via_parte
        rows_without_number.at[index, 'località'] = localita_parte

rows_without_number[['indirizzo', 'località']]

In [ ]:
c = []

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if '(chiareggio)' in str(indirizzo).lower():
        c.append(row)
    if 'teglio' in str(indirizzo).lower():
        c.append(row)
    if 'via turte' in str(indirizzo).lower():
        c.append(row)

df_c = pd.DataFrame(c)

df_c

In [ ]:
# ERROR 4) Some cells in the 'address' column have duplicate information or are formatted incorrectly.

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if '(chiareggio)' in str(indirizzo).lower():
        indirizzo = indirizzo.replace('località pian del lupo (chiareggio)', 'località pian del lupo')
        rows_without_number.at[index, 'info_indirizzo'] = '(chiareggio)'

    if 'via prato valentino teglio' in str(indirizzo).lower():
        indirizzo = indirizzo.replace('via prato valentino teglio','via prato valentino')
        rows_without_number.at[index, 'info_indirizzo'] = 'teglio'

    if 'località san giuseppe' in str(indirizzo).lower():
        indirizzo = indirizzo.replace('località san giuseppe snc san giuseppe', 'località san giuseppe snc')
        rows_without_number.at[index, 'info_indirizzo'] = 'san giuseppe'

    if 'galleria roma- piazza mazzini 8' in str(indirizzo).lower():
        indirizzo = indirizzo.replace('galleria roma- piazza mazzini 8', 'piazza mazzini 8')
        rows_without_number.at[index, 'info_indirizzo'] = 'galleria roma'

    if 'via funivia / zandilla 6' in str(indirizzo).lower():
        indirizzo = indirizzo.replace('via funivia / zandilla 6', 'via zandilla 6')

    if 'via di sotto -' in str(indirizzo).lower():
        indirizzo = indirizzo.replace('via di sotto -', 'via di sotto')


    rows_without_number.at[index, 'indirizzo'] = indirizzo
    rows_without_number.at[index, 'info_indirizzo'] = info_indirizzo

rows_without_number[['indirizzo', 'località', 'info_indirizzo']]

In [ ]:
c = []

for index, row in rows_without_number.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']

    if 'località' in str(indirizzo).lower():
        c.append(row)
df_c = pd.DataFrame(c)

df_c

In [ ]:
rows_without_number[['nome','indirizzo', 'info_indirizzo', 'località', 'comune']]

## Update the starting Dataset named "base" with rows_without_number

In [ ]:
# Creating a temporary DataFrame for selected rows
selected_rows = rows_without_number[['indirizzo', 'info_indirizzo', 'località']]

# Overwrite selected rows in the base DataFrame.
base.update(selected_rows)

base

In [ ]:
# Remove the "-" characters from the 'info_address' column.
base['info_indirizzo'] = base['info_indirizzo'].str.replace('-', '')

# Delete the round brackets in the 'info_address' column.
base['info_indirizzo'] = base['info_indirizzo'].str.replace(r'\(|\)', '', regex=True)

# Delete cells that contain only one numeric group or only two letters.
base = base[~base['info_indirizzo'].str.match(r'^\d+$|^([a-zA-Z]{2})$')]

In [ ]:
base

In [ ]:
d = []

for index, row in base.iterrows():
    info_indirizzo = row['info_indirizzo']

    if 'località' in str(info_indirizzo).lower() or 'loc' in str(info_indirizzo).lower():
        d.append(row)

df_d = pd.DataFrame(d)

df_d

In [ ]:
def sposta_dati(row):
    info_indirizzo = row['info_indirizzo']
    if 'loc' in str(info_indirizzo).lower() or 'località' in str(info_indirizzo).lower():
        if 'semogo' in str(info_indirizzo).lower():
            row['località'] = 'località semogo'
            row['info_indirizzo'] = ''
        elif 'arnoga' in str(info_indirizzo).lower():
            row['località'] = info_indirizzo
            row['info_indirizzo'] = ''
        elif 'bracca' in str(info_indirizzo).lower():
            row['comune'] = 'Bracca'
            row['info_indirizzo'] = ''

    return row

base = base.apply(sposta_dati, axis=1)

righe_non_vuote = base[base['località'].notnull()] # check for non null rows

righe_non_vuote

In [ ]:
lista = ["lanzada", "galizzi", "primolo", "gratacasolo", "trepalle", "cané", "tresenda", "cepina", "isolaccia"]

for index, row in base.iterrows():
    info_indirizzo = row['info_indirizzo']

    for value in lista:
        if value in info_indirizzo.lower():
            base.at[index, 'località'] = info_indirizzo
            base.at[index, 'info_indirizzo'] = ''
            break

base[['indirizzo', 'info_indirizzo', 'località']]

In [ ]:
for index, row in base.iterrows():
    indirizzo = row['indirizzo']
    info_indirizzo = row['info_indirizzo']
    località = row['località']

    if 'condominio ginepro' in indirizzo.lower():
        base.at[index, 'indirizzo'] = 'via pradosole'
        base.at[index, 'info_indirizzo'] = 'condominio ginepro'

    if 'località' in località.lower():
        base['località'] = base['località'].str.replace('località', '')
        base['località'] = base['località'].str.replace('Località', '')

    if 'frazione' in località.lower():
        base['località'] = base['località'].str.replace('frazione', '')

base[['indirizzo', 'info_indirizzo', 'località']]

In [ ]:
base

In [ ]:
# Restore the index of the DataFrame.
base = base.reset_index(drop=True)

In [ ]:
# Saving the DataFrame to a CSV file.
base.to_csv('booking_base.csv', index=False)

In [ ]:
# Saving the DataFrame to a JSON file.
base.to_json('booking_base.json', orient='records')

In [ ]:
booking_structure = pd.read_csv('booking_base.csv', encoding='utf-8', header=0)
# Print the number of rows and columns of the file "booking_base.csv"
print("Numero di righe di booking_base.csv:", booking_structure.shape[0])
print("Numero di colonne di booking_base.csv:", booking_structure.shape[1])

In [ ]:
# Delete duplicates from the DataFrame
booking_structure = booking_structure.drop_duplicates()

print("Numero di righe di booking_base.csv:", booking_structure.shape[0])
print("Numero di colonne di booking_base.csv:", booking_structure.shape[1])

# Saving the DataFrame in 'booking_base.csv' without duplicates.
booking_structure.to_csv('booking_base.csv', index=False)